# `lunapi` tutorial

Links to notebooks in this repository: [Index](./00_overview.ipynb) | [Luna tutorial](./tutorial.ipynb) | 
[Individuals](./01_indivs.ipynb) | [Projects](./02_projects.ipynb) | [Staging](./03_staging.ipynb) | [Models](./04_models.ipynb) | [Advanced](./98_advanced.ipynb) | [Reference](./99_reference.ipynb)

---

This notebook steps through the original Luna [tutorial](https://zzz.bwh.harvard.edu/luna/tut/tut1/) using the Python-based `lunapi` instantiation of the Luna library.  Below we will follow, more or less step-by-step, each part to introduce a typical workflow using `lunapi`.   For further details on `lunapi` see the other [introductory and reference pages](./00_overview.ipynb) in this repository.  Even if you don't plan on using the command-line version of Luna, it is worth skimming those first three tutorial pages, as they give a little more context on the goals of the tutorial exercises. 

# Part 1

This part follows the steps described [here](https://zzz.bwh.harvard.edu/luna/tut/tut1/).  Rather than download the tutorial data as directed on that page, here we assume you are either using the Docker version of `lunapi`, or you've cloned the [luna-api-notebooks](https://github.com/remnrem/luna-api-notebooks) repository, both of which contain a folder `tutorial/` with all the necessary data and files. We also assume you are using Jupyterlab to open and interact with this notebook.

## Setting up a project

The first steps are to import the `lunapi` module and create a single, new project (always called `proj` by convention here):

In [1]:
import lunapi as lp

In [2]:
proj = lp.proj()

initiated lunapi v1.2.3 <lunapi.lunapi0.luna object at 0xffff7c88f8b0> 



We'll then load the premade [_sample-list_](https://zzz.bwh.harvard.edu/luna/luna/args/#sample-lists) into this project, which is a plaintext file `s.lst` within the `tutorial/` folder:

In [3]:
proj.sample_list( 'tutorial/s.lst' )

read 3 individuals from tutorial/s.lst


This points to three individuals, each with an EDF and an XML annotation file containing prior staging information:

In [4]:
proj.sample_list()

,ID,EDF,Annotations
1,learn-nsrr01,tutorial/edfs/learn-nsrr01.edf,{tutorial/edfs/learn-nsrr01.xml}
2,learn-nsrr02,tutorial/edfs/learn-nsrr02.edf,{tutorial/edfs/learn-nsrr02.xml}
3,learn-nsrr03,tutorial/edfs/learn-nsrr03.edf,{tutorial/edfs/learn-nsrr03.xml}


## Displaying EDF files

The `desc()` function is a convenient way to get summary information on all individuals in a project (this is the equivalent of the Luna `DESC` command): 

In [5]:
proj.desc()

,ID,Gapped,Date,Start(hms),Stop(hms),Dur(hms),Dur(s),# sigs,# annots,Signals
1,learn-nsrr01,N,01.01.85,21.58.17,09.20.17,11:22:00,40920,14/14,10,SaO2[1] PR[1] EEG_sec[125] ECG[250] EMG[125] EOG_L[50] EOG_R[50] EEG[125] AIRFLOW[10] THOR_RES[10] ABDO_RES[10] POSITION[1] LIGHT[1] OX_STAT[1]
2,learn-nsrr02,N,01.01.85,21.18.06,07.15.36,09:57:30,35850,14/14,10,SaO2[1] PR[1] EEG_sec[125] ECG[250] EMG[125] EOG_L[50] EOG_R[50] EEG[125] AIRFLOW[10] THOR_RES[10] ABDO_RES[10] POSITION[1] LIGHT[1] OX_STAT[1]
3,learn-nsrr03,N,01.01.85,20.15.00,07.37.00,11:22:00,40920,14/14,10,SaO2[1] PR[1] EEG_sec[125] ECG[250] EMG[125] EOG_L[50] EOG_R[50] EEG[125] AIRFLOW[10] THOR_RES[10] ABDO_RES[10] POSITION[1] LIGHT[1] OX_STAT[1]


<div class="alert alert-block alert-info"><b>Note:</b> Note that whereas most Luna commands are evaluated via the `proc()` or `eval()` functions, this is a special case as the      Luna <tt>DESC</tt> command returns it output directy to the console (rather than Luna's formal output mechanism).  As such, we created the above wrapper.</div>
   
Next, we'll pull out the first individual from the project sample list, via the `inst()` function and assign this record to the variable `p` (by convention, we always label the primary individual under consideration as `p`):

In [6]:
p = proj.inst( 1 )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT


We can get a simple list of the channels, here returned as a Pandas dataframe (which naurally mirrors the information printed in the console/log above). This can be used programatically, e.g. saved as an object, etc.

In [7]:
p.channels()

,Channels
0,SaO2
1,PR
2,EEG_sec
3,ECG
4,EMG
5,EOG_L
6,EOG_R
7,EEG
8,AIRFLOW
9,THOR_RES


We can do likewise with any attached annotations by using `annots()` (in this case from the XML file):

In [8]:
p.annots()

,Annotations
0,Arousal
1,Hypopnea
2,N1
3,N2
4,N3
5,Obstructive_Apnea
6,R
7,SpO2_artifact
8,SpO2_desaturation
9,W


Below we'll see how to get more information on channels and annotations, using the `HEADERS` and `ANNOTS` commands respectively.

## Label remapping and sanitization

As discussed in the original tutorial, by default Luna will _sanitize_ signal and annotation labels, with the aim of making them easier to work with downstream, e.g. by removing spaces and special characters.   Thus, the annotation above `Obstructive_Apnea` is actually `Obstructive Apnea` in the original XML, but Luna has modified it when reading it in (all intra-string special characters are reduced to underscores).   

We can turn off this default behavior by setting the special variables `sanitize` and `keep-spaces` using the project-level `vars()` function.   See [this page](https://zzz.bwh.harvard.edu/luna/luna/args/) for details on the available options.  Here we supply a `dict` of option/value pairs: i.e. note the Python syntax of `{ 'key': value, 'key':value }`.

In [9]:
proj.vars( { 'sanitize':'F' , 'keep-spaces':'T' } )

Note that these particular options need to be set _before_ attaching an individual via `inst()`, as any mappings occur when first attaching a new individual instance.  Thus, as we've changed these options, we need to re-run the call to `inst()` to see any changes:

In [10]:
p = proj.inst( 1 ) 
p.channels()

___________________________________________________________________
Processing: learn-nsrr01 | tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG(sec) | ECG | EMG | EOG(L) | EOG(R) | EEG
  AIRFLOW | THOR RES | ABDO RES | POSITION | LIGHT | OX STAT


,Channels
0,SaO2
1,PR
2,EEG(sec)
3,ECG
4,EMG
5,EOG(L)
6,EOG(R)
7,EEG
8,AIRFLOW
9,THOR RES


In [11]:
p.annots()

,Annotations
0,Arousal ()
1,Hypopnea
2,N1
3,N2
4,N3
5,Obstructive Apnea
6,R
7,SpO2 artifact
8,SpO2 desaturation
9,W


In [12]:
proj.desc()

,ID,Gapped,Date,Start(hms),Stop(hms),Dur(hms),Dur(s),# sigs,# annots,Signals
1,learn-nsrr01,N,01.01.85,21.58.17,09.20.17,11:22:00,40920,14/14,10,SaO2[1] PR[1] EEG(sec)[125] ECG[250] EMG[125] EOG(L)[50] EOG(R)[50] EEG[125] AIRFLOW[10] THOR RES[10] ABDO RES[10] POSITION[1] LIGHT[1] OX STAT[1]
2,learn-nsrr02,N,01.01.85,21.18.06,07.15.36,09:57:30,35850,14/14,10,SaO2[1] PR[1] EEG(sec)[125] ECG[250] EMG[125] EOG(L)[50] EOG(R)[50] EEG[125] AIRFLOW[10] THOR RES[10] ABDO RES[10] POSITION[1] LIGHT[1] OX STAT[1]
3,learn-nsrr03,N,01.01.85,20.15.00,07.37.00,11:22:00,40920,14/14,10,SaO2[1] PR[1] EEG(sec)[125] ECG[250] EMG[125] EOG(L)[50] EOG(R)[50] EEG[125] AIRFLOW[10] THOR RES[10] ABDO RES[10] POSITION[1] LIGHT[1] OX STAT[1]


We'll reset these options for the rest of this tutorial however, i.e. to keep Luna's default approach to adjusting labels:

In [13]:
proj.vars( { 'sanitize':'T' , 'keep-spaces':'F' } )

## Signal summary statistics

The next step of the tutorial involves calculating some simple signal summary statistics (e.g. mean, RMS, etc).  This introduces two key concepts:  

   * how to apply one or more [Luna commands](https://zzz.bwh.harvard.edu/luna/ref/) to one or more individuals
   * and how to work with the output mechanism to retrieve the results.

Here we start by looking at a single individual (the first person from the sample list, which we've already associated with the variable `p` above).  Any valid Luna script/commands (here `STATS`) can be passed via the `eval()` (or equivalently `proc()`) functions.   

In [14]:
p.eval( 'STATS' )

 ..................................................................
 CMD #1: STATS
   options: sig=*
  processing: SaO2 PR EEG(sec) ECG EMG EOG(L) EOG(R) EEG AIRFLOW THOR RES ABDO RES POSITION LIGHT OX STAT


,Command,Strata
0,STATS,CH


The output above is the same as you'd see on the console if running command-line Luna.  The `eval()` function also returns a table (Pandas dataframe) that mirrors what the command-line `destrat` tool would report if querying the output generated here.  There is only a single table of results, which is _stratified_ by channel (`CH`).  That is, each variable (e.g. `RMS`) can have multiple values, each associated with a different channel.  

After running `eval()` or `proc()` for an individual, any output is stored in that individual's _result cache_ (but note, this is overwritten by each new call of `eval()` or `proc()`).   To see the tables in the cache at any time, you can explicitly call the `strata()` function:

In [15]:
p.strata()

,Command,Strata
0,STATS,CH


To extract a table of results (as a Pandas dataframe), you can use `table()`, passing it the command and strata labels, i.e. the values reported in the rows of `strata()`:

In [16]:
p.table( 'STATS' , 'CH' )

,ID,CH,KURT,MAX,MEAN,MIN,P01,P02,P05,P10,...,P60,P70,P80,P90,P95,P98,P99,RMS,SD,SKEW
0,learn-nsrr01,ABDO RES,5.26486,0.992157,0.021829,-1.0,-0.709804,-0.505882,-0.317647,-0.2,...,0.027451,0.05098,0.098039,0.262745,0.435294,0.662745,0.87451,0.235721,0.234708,0.334763
1,learn-nsrr01,AIRFLOW,20.490936,0.992157,0.066442,-1.0,-0.2,-0.129412,-0.066667,-0.027451,...,0.07451,0.090196,0.113725,0.160784,0.207843,0.278431,0.34902,0.121956,0.102268,0.138384
2,learn-nsrr01,ECG,13.984303,1.25,0.009396,-1.240196,-1.240196,-0.857843,-0.191176,-0.083333,...,0.02451,0.034314,0.034314,0.083333,0.210784,0.828431,1.25,0.266661,0.266495,-0.115245
3,learn-nsrr01,EEG,4.657487,125.0,-0.301199,-124.019608,-124.019608,-124.019608,-70.098039,-24.019608,...,2.45098,4.411765,8.333333,23.039216,72.058824,125.0,125.0,37.801351,37.800154,0.086212
4,learn-nsrr01,EEG(sec),4.703039,125.0,1.185578,-124.019608,-124.019608,-124.019608,-78.921569,-20.098039,...,3.431373,6.372549,10.294118,22.058824,78.921569,125.0,125.0,40.325406,40.307978,-0.068798
5,learn-nsrr01,EMG,2.562353,31.5,-6.855696,-31.252941,-31.252941,-31.252941,-31.252941,-31.252941,...,-4.076471,-3.335294,-2.594118,-0.370588,5.311765,31.5,31.5,13.970074,12.172199,0.033768
6,learn-nsrr01,EOG(L),1.926088,125.0,0.472551,-124.019608,-124.019608,-124.019608,-124.019608,-61.27451,...,2.45098,4.411765,8.333333,52.45098,125.0,125.0,125.0,54.461877,54.45984,-0.026404
7,learn-nsrr01,EOG(R),2.016313,125.0,0.321447,-124.019608,-124.019608,-124.019608,-124.019608,-56.372549,...,2.45098,4.411765,7.352941,46.568627,125.0,125.0,125.0,54.009293,54.00835,-0.024934
8,learn-nsrr01,LIGHT,360.359887,1.0,0.997263,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.998631,0.052246,-19.035055
9,learn-nsrr01,OX STAT,-0.063555,2.0,0.428886,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,2.0,2.0,2.0,0.926107,0.820821,1.391434


This displays the whole table: you can alternatively copy it to a Python object - here we'll typically use `t` to refer to the current result table under consideration.  You can naturally use the array of Python/Pandas functionality to manipulate those results: e.g. extratcing a sub-table of the mean for just three channels:

In [17]:
t = p.table( 'STATS' , 'CH' )
t.query( "CH in ['SaO2','ECG','EMG']")[ ['CH','MEAN'] ]

,CH,MEAN
2,ECG,0.009396
5,EMG,-6.855696
12,SaO2,76.924249


The `lunapi` module also provides a simple wrapper function `lp.subset()` to subset table rows i) by individual or ii) by the values of factors and/or other variables, as well as iii) subsetting columns to particular variables. Here are some (unrealistic) examples (designed simply to show the syntax):

In [18]:
lp.subset( t , vars = [ 'CH' , 'RMS' ] , qry = " RMS > 50" )

,ID,CH,RMS
6,learn-nsrr01,EOG(L),54.461877
7,learn-nsrr01,EOG(R),54.009293
11,learn-nsrr01,PR,64.952329
12,learn-nsrr01,SaO2,85.566543


In [19]:
lp.subset( t , qry = " CH == 'PR' " )

,ID,CH,KURT,MAX,MEAN,MIN,P01,P02,P05,P10,...,P60,P70,P80,P90,P95,P98,P99,RMS,SD,SKEW
11,learn-nsrr01,PR,2.311002,200.0,57.348522,0.201419,0.201419,0.201419,0.201419,0.201419,...,68.36347,70.316625,72.26978,75.199512,79.301137,89.262226,147.261769,64.952329,30.495489,-0.457502


In [20]:
# using f-string query to pull a list of channels
chs = [ 'PR', 'POSITION' ] 
lp.subset( t , vars = [ 'CH' , 'MEAN' , 'RMS' ] , qry = f"CH in {chs}" )

,ID,CH,MEAN,RMS
10,learn-nsrr01,POSITION,1.654423,1.915741
11,learn-nsrr01,PR,57.348522,64.952329


## Working with annotations

Next, the tutorial considers working with associated annotation data.   We can get a summary of each annotation _class_ by the `ANNOTS` command, as described [here](https://zzz.bwh.harvard.edu/luna/ref/annotations/#annots).   Note that here we are applying `proc()` to the entire sample list (`proj`) rather than a single individual.  This means the Luna commands will be evaluated for all three individuals in the sample list sequentially:

<div class="alert alert-block alert-info"><b>Note:</b> This particular analysis runs quickly, but note that for non-trivial samples and analyses, this could take some time.  Also, currently the Python version of Luna does not show output until the entire job is complete.   It is therefore advisable to test scripts on one or two individuals first, before running on a larger set.    Alternatively, the command-line version of Luna is in some ways better suitable to dealing with large (e.g. 1000s of records), especially when combined with a compute cluster.  In any case, in the future we'll add better support for e.g. progress reports, and for better ability to manually halt long-running jobs.</div>


In [21]:
res = proj.proc( 'ANNOTS' )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: ANNOTS
   options: sig=*
  set epochs to default 30 seconds, 1364 epochs
  keeping annotations based on any overlap with an unmasked region
___________________________________________________________________
Processing: learn-nsrr02 | tutorial/edfs/learn-nsrr02.edf
 duration 09.57.30, 35850s | time 21.18.06 - 07.15.36 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ................................................

Note that here we use `proc()` instead of `eval()`.  When working at the project level (i.e. `proc.proc()`), Luna commands can only be evaluated with `proc()`.  When working at the individual level (i.e. `p.proc()`), then these commands are basically interchangeable, with the exception that `p.proc()` returns the results (i.e. here stored in `res`) as well as caching them internally, whereas `p.eval()` only returns the `strata()` summary table.  

In any case, here we need to call `proj.strata()` explicitly to see what was generated:

In [22]:
proj.strata()

,Command,Strata
0,ANNOTS,ANNOT
1,ANNOTS,ANNOT_INST
2,ANNOTS,ANNOT_INST_T1_T2


We'll pull the count and duration (totial seconds spanned) of each annotation class - noting that we get output for all three individuals:

In [23]:
proj.table( 'ANNOTS' , 'ANNOT' )

,ID,ANNOT,COUNT,DUR
0,learn-nsrr01,Arousal,194,1713.5
1,learn-nsrr02,Arousal,51,644.8
2,learn-nsrr03,Arousal,282,4002.4
3,learn-nsrr03,Central_Apnea,1,12.3
4,learn-nsrr01,Hypopnea,361,7012.6
5,learn-nsrr02,Hypopnea,101,1536.9
6,learn-nsrr03,Hypopnea,181,3215.2
7,learn-nsrr01,N1,109,3270.0
8,learn-nsrr02,N1,11,330.0
9,learn-nsrr03,N1,52,1560.0


As above, we can subset this table to pull out quantities of intersest.  To mirror the original tutorial, we want the count of obstructive apnea annotations per person:

In [24]:
t = proj.table( 'ANNOTS' , 'ANNOT' )
lp.subset( t , vars = ['ANNOT','COUNT'] , qry = " ANNOT == 'Obstructive_Apnea' " )

,ID,ANNOT,COUNT
16,learn-nsrr01,Obstructive_Apnea,37
17,learn-nsrr02,Obstructive_Apnea,5
18,learn-nsrr03,Obstructive_Apnea,163


Happily, this gives the same results as the original tutorial.   

The next step was to extract only REM-specific event counts.  This will involve re-running the `ANNOTS` command but also including a [`MASK`](https://zzz.bwh.harvard.edu/luna/ref/masks) to first restrict to REM (stage annotation `R`) events:

In [25]:
res = proj.proc( 'MASK ifnot=R & ANNOTS' )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 14 (of 14) selected in a standard EDF file
  SaO2 | PR | EEG_sec | ECG | EMG | EOG_L | EOG_R | EEG
  AIRFLOW | THOR_RES | ABDO_RES | POSITION | LIGHT | OX_STAT
 ..................................................................
 CMD #1: MASK
   options: ifnot=R sig=*
  set epochs, to default length 30, 1364 epochs
  set masking mode to 'force'
  annots: R
  applied annotation mask for 1 annotation(s)
  238 epochs match; 1126 newly masked, 0 unmasked, 238 unchanged
  total of 238 of 1364 retained
 ..................................................................
 CMD #2: ANNOTS
   options: sig=*
  keeping annotations based on any overlap with an unmasked region
___________________________________________________________________
Processing: learn-nsrr02 | tutorial/edfs/learn-nsrr02.e

Re-extracting the same metrics from the output, we again get similar results as the tutorial.  Note that the thrird individual does not have any scored REM sleep, and so there is no value for that individual.

In [26]:
t = proj.table( 'ANNOTS' , 'ANNOT' )
lp.subset( t , vars = ['ANNOT','COUNT'] , qry = " ANNOT == 'Obstructive_Apnea' " )

,ID,ANNOT,COUNT
4,learn-nsrr01,Obstructive_Apnea,27
5,learn-nsrr02,Obstructive_Apnea,3


## Putting it together

Having run one or two Luna commands and looked at some output, we can now move to more involved Luna scripts.  Here we also use a _variable_ in the script. This allows for different individuals in the sample-list or different runs of the same script to implement slight modifications, e.g. looking at different channels, or different sleep stages.  This script is stored in the file `first.txt`: the `lp.cmdfile()` utility reads this in, and does any necessary pre-processing ( e.g. removing comments, etc) and returns a string that can be fed into `proc()` or `eval()`:

In [27]:
lp.cmdfile( 'tutorial/cmd/first.txt' )

'EPOCH len=30 & TAG tag=STAGE/${stage} & MASK ifnot=${stage} & RESTRUCTURE & STATS sig=EEG'

This script defines 30-second epochs, sets an output `TAG` to keep track of the sleep stage (which here is defined by the variable `${stage}` [note - this could be any valid variable name, e.g. `${s}`, etc]. It then removes all other epochs for other stages, internally restructures the data, then generates signal statistics for a single channel `EEG`.  In order to avoid Luna complaining that the `${stage}` variable has not been defined, we first need to define it.  When using the command-line version, this would be passed in as a command-line option: e.g. something like this to define `${stage}`:

For a single stage, we can achieve the same result by using `proj.vars()` [or equivalently, `proj.var()`].  (We'll also use this to set `sig` to `EEG` so that only this channel is loaded in, as only that signal is used - this approach can speed things up in very large samples):

In [28]:
proj.var( { 'stage': 'N1', 'sig': 'EEG' } )

We'll then use `proj.proc()` to run this script: Luna will swap out instances of `${stage}` with the text `N1` before running the commands:

In [29]:
res = proj.proc( lp.cmdfile( 'tutorial/cmd/first.txt' ) )

___________________________________________________________________
Processing: learn-nsrr01 | tutorial/edfs/learn-nsrr01.edf
 duration 11.22.00, 40920s | time 21.58.17 - 09.20.17 | date 01.01.85

 signals: 1 (of 14) selected in a standard EDF file
  EEG
 ..................................................................
 CMD #1: EPOCH
   options: len=30 sig=*
  set epochs, length 30 (step 30, offset 0), 1364 epochs
 ..................................................................
 CMD #2: TAG
   options: sig=* tag=STAGE/N1
  setting analysis tag to [STAGE/N1]
 ..................................................................
 CMD #3: MASK
   options: ifnot=N1 sig=*
  set masking mode to 'force'
  annots: N1
  applied annotation mask for 1 annotation(s)
  109 epochs match; 1255 newly masked, 0 unmasked, 109 unchanged
  total of 109 of 1364 retained
 ..................................................................
 CMD #4: RESTRUCTURE
   options: sig=*
  restructuring as an EDF+:  

Looking at the table of results with `strata()` - of particular interest is the output of the `STATS` command, which is stratified by channel (`CH`) but now also by sleep stage (`STAGE`) as a consequence of the `TAG` command:

In [30]:
proj.strata()

,Command,Strata
0,EPOCH,BL
1,EPOCH,STAGE
2,MASK,EMASK_STAGE
3,RESTRUCTURE,STAGE
4,STATS,CH_STAGE


We can review all outputs (stored in `res`) with `lp.show()`. (Note that in real contexts, these tables could become very large, and so `lp.show()` would not be the preferred way to extract results.)

In [31]:
lp.show( res )

EPOCH: BL


/opt/conda/lib/python3.12/site-packages/lunapi/lunapi1.py:2421: SyntaxWarning: invalid escape sequence '\.'
  if re.search('\.edf\.gz$',self.curr_edf,re.IGNORECASE) or re.search('\.edfz$',self.curr_edf,re.IGNORECASE):
/opt/conda/lib/python3.12/site-packages/lunapi/lunapi1.py:2421: SyntaxWarning: invalid escape sequence '\.'
  if re.search('\.edf\.gz$',self.curr_edf,re.IGNORECASE) or re.search('\.edfz$',self.curr_edf,re.IGNORECASE):


AttributeError: module 'IPython.core.display' has no attribute 'display'

Alternatively, we can extract only a particular table of interest:

In [ ]:
proj.table('STATS','CH_STAGE')

Next, we'll set up a simple Python loop to run this script on all individuals, once for each sleep stage of N1, N2, N3 and REM.  We'll store the output of each `proc()` in a Python dictionary called `res`.  To reduce the clutter to the screen, we'll use `silent_proc()` instead of `proc()` - this is indentical except for the console output produced.   (In general, only use this when you are sure a command will run as expected.)

In [ ]:
res = { } 
stages = [ 'N1', 'N2', 'N3', 'R' ]
for stage in stages:
    print( "processing stage " , stage )
    proj.var( 'stage', stage )
    res[ stage ] = proj.silent_proc( lp.cmdfile( 'tutorial/cmd/first.txt' ) )

We can then pull out the results for particular stages: e.g. here directly accessing the tables in `res`:

In [ ]:
res.keys()

In [ ]:
res[ 'N1' ].keys()

In [ ]:
res[ 'N1' ][ 'STATS: CH_STAGE' ]

In [ ]:
res[ 'N2' ][ 'STATS: CH_STAGE' ]

When the output is in this format (i.e. split across conditions/stages), it is often convenient to pull the similar tables together for analysis, etc.  For example, if we want one table with all individuals and all stages, we can use built-in Pandas options, such as `concat()`:  

In [ ]:
import pandas as pd
pd.concat( [ res[ stage ][ 'STATS: CH_STAGE' ][ ['ID','STAGE','MEAN'] ] for stage in stages ] )

Alternatively, `lunapi` has a simple wrapper function to do this (assuming the same type of structure of nested-dictionaries in the output).  It can also add an index to reflect the higher-level (here sleep stage, which is also present in the original tables, but this can be useful if it isn't).  This takes all tables with the key `RESTRUCTURE: STAGE` from `res` and concatenates them, adding an index `IDX` based on `res.keys()` (i.e. in this example, sleep stages):

In [ ]:
lp.concat( res , 'RESTRUCTURE: STAGE'  , add_index = 'IDX' )

## Using data freezes

Alternatively, rather than using Python loops to produce results for different stages, we could have (in this particular scenario) achieved the same goals by using Luna's [freeze/thaw](https://zzz.bwh.harvard.edu/luna/ref/freezes) mechanism to derive the various sub-analyses from a single run for each individual, as shown in the script `tutorial/cmd/first-freezes.txt`:

In [ ]:
%%sh
cat tutorial/cmd/first-freezes.txt

In [ ]:
res = proj.proc( lp.cmdfile( 'tutorial/cmd/first-freezes.txt' ) )

Here we used `proc()` instead of `silent_proc()` as above, so we've got a lot of output.  Note that final message about skipping the `STATS` command: this is fine -- recall that there were truly no REM records for this third individual.  

Looking at the cached outputs (i.e. this contains the same information as in `res`):

In [ ]:
proj.strata()

We can extract the same table of RMS values by sleep stage, getting the same answers as in the original tutorial:

In [ ]:
proj.table( 'STATS' , 'CH_STAGE' )[ ['ID','STAGE','RMS' ] ] 

We can get a slightly more convenient format of output with `pd.pivot()`:

In [ ]:
t = proj.table( 'STATS' , 'CH_STAGE' )
pd.pivot( t , index = 'ID' , columns = 'STAGE' , values='RMS' )

# Part 2

## General syntax notes

We've seen how to apply Luna commands (via `proc()` at the project-level, or either `proc()` or `eval()` at the instance-level), as well as setting variables/options via `proj.var()` and working with output via `strata()`, `table()`, `lp.show()` and other utilties.  

To restate some of the equivalences here, first __when working with a single individual__ (e.g. `learn-nsrr02`).  Consider if this is the command-line Luna script, to get RMS per channel: 

```
luna s.lst id=learn-nsrr02 a=1 b=xyz @param.txt -o out.db -s HEADERS 

destrat out.db +HEADERS -r CH
```
(Note that we set a variables `${a}` and `${b}` and include other information from a [_parameter-file_](https:/zzz.bwh.harvard.edu/luna/luna/args/#parameter-files) - these are not used in the command, but just included here to show the corresponding syntax for these things in `lunapi`).   In `lunapi`, we'd do the following steps (assuming `proj = lp.proj()` has already been called):

```
proj.sample_list( 's.lst' )
proj.var( { 'a': 1 , 'b': 'xyz' } )     # n.b. using dict as an inputs to specify >1 variable at once
proj.include( 'param.txt' )
p = proj.inst( 'learn-nsrr02' )         # n.b. using ID-string instead of a row number (2 in this case)
p.eval( 'HEADERS' )
p.table( 'HEADERS' , 'CH' )
```

Second, __when working at the project/sample-list level__, the original Luna command may look very similar (simply dropping the `id`):

```
luna s.lst a=1 b=xyz @param.txt -o out.db -s HEADERS 

destrat out.db +HEADERS -r CH
```

When using `lunapi`, we'd achieve similar results as follows:

```
proj.sample_list( 's.lst' )
proj.var( { 'a': 1 , 'b': 'xyz' } )
proj.include( 'param.txt' )
proj.proc( 'HEADERS' )                  # i.e. call to proj.proc() rather than p.eval()
proj.table( 'HEADERS' , 'CH' )          # i.e. a call to proj.table() rather than p.table()
```


Please see the [reference](./99_reference.ipynb) pages for more details.

## Parameter files

In the above example, we introduced a new function: `include()`, which simply reads tab-separated key/value pairs from a text file and assigns those variables those values (using `proj.vars()`). The `vars.txt` file is as follows:

In [ ]:
%%sh
cat tutorial/cmd/vars.txt

We can include these defintions as follows - i.e. equivalent to running `proj.vars( 'myepoch' , 10 )` etc.  The `include()` function returns the number of variables successfully set:

In [ ]:
proj.include( 'tutorial/cmd/vars.txt' )

We can review _all_ attached project-level variables using `proj.vars()` - note that as well as the variables specified explicitly above, this will include any pre-defined variables that Luna uses internally (i.e. here, the grouping of EEG variables) - these internal variables can be ignored (or changed) as desired):

In [ ]:
proj.vars()

With these variable set, we can now run the `second.txt` script to mirror the original tutorial, which ran this for the first individual:

Here is the script:

In [ ]:
lp.cmdfile( 'tutorial/cmd/second.txt' )

And running this for the first individual:

In [ ]:
p = proj.inst( 'learn-nsrr01' )
p.eval( lp.cmdfile( 'tutorial/cmd/second.txt' ) )

The above output now contains the NREM (N1,N2 and N3) values for two EEG channels for the basic stats, defined on 10-second epochs:

In [ ]:
p.table( 'STATS' , 'CH' )[ ['ID','CH','RMS'] ]

# Part 3

In this final section of the tutorial, we'll use Luna to calculate some typical metrics of interest for sleep studies, including:

   * how basic statistics vary across the night
   * measures of sleep macro-architecture
   * filtering and artifact detection in the EEG
   * spectral analysis using fast Fourier transforms
   * spindle detection using wavelets


## Epoch-level summaries

Returning to the `STATS` command described above, here we use it with the epoch option, in order to generate per-epoch statistics for the second individual in the sample-list.  Because we previosuly set `sig` to only load EEGs, we want to clear it now (i.e. otherwise the EMG and ECG signals will not be available).   That is - project-level options persist across different calls to `proc()` etc, and so be careful to keep them in track.

In [ ]:
proj.var( 'sig' )

In [ ]:
proj.var( 'sig', '') 

In [ ]:
p = proj.inst( 2 )
p.eval( 'EPOCH & STATS sig=EMG,ECG epoch' )

As well as being applied only to the second EDF, this generates statistics only for the ECG and EMG signals. After defining 30-second epochs (the default from the EPOCH command), it generates basic statistics per-epoch (the epoch option for the STATS command), as well as whole-signal summaries. Note that 1195 epochs are generated, which corresponds to the stated duration of 9 hours, 57 and a half minutes (i.e. ( 9 * 60 + 57 ) * 2 + 1 = 1195).

The overall goal is to generate a plot of per-epoch RMS for the ECG. We can do that by extracting a table of _epoch-level_ RMS values:

In [ ]:
p.table( 'STATS' , 'CH_E' )

In [ ]:
d = p.table( 'STATS' , 'CH_E' )
d = d[ d['CH'] == 'ECG' ][ [ 'E', 'RMS' ] ] 

To make a basic plot of epoch-level ECG RMS:

In [ ]:
import matplotlib.pyplot as plt
plt.plot( d[ 'E' ] , d[ 'RMS' ] )
plt.xlabel('Epoch (N)')
plt.ylabel('ECG RMS')
plt.show()

We can also extract the sleep stage labels, which align with the above-defined epochs, using the convenience `stages()` command:

In [ ]:
stages = p.stages()
len( stages )

Following the tutorial, we'll only plot epochs 1 to 950 in order to exclude the outliers at the end of the study (during wake / after recording stopped).  We'll use the stage labels to color the values for each epoch (W,R,N = green, red, blues):

In [ ]:
d[ 'SS' ] = stages[ 'STAGE' ]
d = d.loc[ d['E'] <= 950 , ] 
plt.scatter(d['E'], d['RMS'], c = lp.stgcol( d[ 'SS' ] ) , s=5 )
plt.show()

## Hypnograms

Next, we extract hypnogram-based statistics using the `HYPNO` command, here running for all individuals:

In [ ]:
res = proj.proc( 'HYPNO epoch' )

Here we extract the stage durations for each individual (note: this includes some pre-defined stage labels including `N4` which can be ignored here):

In [ ]:
t = proj.table( 'HYPNO' , 'SS' )
pd.pivot( t , index = 'ID' , columns = 'SS' , values='MINS' )

Next we extract some cycle-based statistics:

In [ ]:
t = proj.table( 'HYPNO' , 'C' )
t = t.astype( {"C": int } )   # makes the table format nicer
pd.pivot( t , index = 'ID' , columns = 'C' , values=[ 'NREMC_START','NREMC_MINS'] )

Finally, we get extract epoch-level statistics, for just for one individual.  All these metrics are described in detail in the [main Luna pages](https://zzz.bwh.harvard.edu/luna/ref/hypnograms/).

In [ ]:
t = proj.table( 'HYPNO', 'E' )
t.loc[ t['ID'] == 'learn-nsrr01' , ['E','STAGE','FLANKING','CLOCK_TIME'] ]

## Epoch-level masks

To illustrate applying different epoch-level masks, consider the following (somewhat unrealistic) example - to select epochs that:

   * occur between 11pm and 3am
   * are in persistent sleep as defined above (i.e. at least 10 minutes of sleep prior)
   * are during stage 2 NREM sleep
   * do not contain any apnea or hypopnea events

The following Luna command script (`cmd/third.txt`) shows one way of doing this, leveraging the ability of `HYPNO` to add annotations on-the-fly (to define persistent sleep):

In [ ]:
lp.cmdfile( 'tutorial/cmd/third.txt' )

We'll run this only for the first individual, as in the original tutorial:

In [ ]:
p = proj.inst(1)
p.eval( lp.cmdfile( 'tutorial/cmd/third.txt' ) )

Following the logic described in the [original tutorial](https://zzz.bwh.harvard.edu/luna/tut/tut3/#epoch-level-masks), we end up with 86 epochs following these steps.  The output of `MASK` above shows this to be the case here too. We can also pull this out explicitly - this shows all the mask steps, but from the script we know the final one if based on the apnea/hypopnea definitions - we see `N_RETAINED` equals 86:

In [ ]:
p.table( 'MASK', 'EMASK' )

## Manipulating EDFs

Here we give an overview of filtering, resampling, relabelling, re-referencing and re-writing signal data. Specifically, we want to:

   * set EEG units to millivolts
   * resample both EEGs to 100 Hz
   * bandpass filter the signals between 0.3 and 35 Hz
   * re-reference EEG2 to EEG1 (i.e. EEG2 = EEG2 - EEG1)
   * write only the EEGs as a new EDF in the folder `newedfs/`

The following command script (`cmd/fourth.txt`) illustrates the above features: 

In [ ]:
%%sh
cat tutorial/cmd/fourth.txt

The original Luna script pulls out only the two EEGs, using this innovocation:

Here, we'll make sure the `sig` variable is set such that only the EEGs are read in.  We'll start by setting it to be blank, and then use the alias terms `EEG1` and `EEG2` (see below):

In [ ]:
proj.vars('sig','')
proj.vars('sig','EEG1,EEG2')
proj.vars('sig')

We'll also read the options defined in the parmeter file `vars.txt`, which include setting aliases for the two EEGs (to `EEG1` and `EEG2`):

In [ ]:
proj.include( 'tutorial/cmd/vars.txt' )

This means that we can use `EEG1` and `EEG2` in subsequent scripts; also, all outputs will be generated using these labels rather than the originals.  Note that although the EDFs do not contain labels `EEG1` or `EEG2` these were previously set as _aliases_ (via the included `vars.txt` file) and so we can use them here.  This is convenient when you have a many-to-one mapping of actually labels and the preferred term (as often happens across and within NSRR datasets, for example).  

To see what the current alias mapping table looks like (i.e. that will be applied when first attaching a study), we can use `proj.aliases()`.  Note that this may include a number of predefined aliases (typically different sleep stage naming conventions) alongside any user-defined ones:

In [ ]:
proj.aliases()

We can now run the script to generate the new EDFs:

In [ ]:
res = proj.proc( lp.cmdfile( 'tutorial/cmd/fourth.txt' ) )

This also generates a new sample list `new.lst` (appending the new files on the end, if it already existed).  We can attach that and see the details of the newly-generated EDFs:

In [ ]:
proj.sample_list( 'new.lst' )

In [ ]:
proj.sample_list()

In [ ]:
proj.desc()

As shown above, this has generated three new EDFs in the `newedfs` folder.  Note that they have the preferred alias labels swapped in too.

## Artifact detection

Here we run a very simple artifact detection pipeline, appropriate for typical limited-montage sleep-EEG studies.  

To see a more advance EEG pipeline, see [this vignette](https://zzz.bwh.harvard.edu/luna/vignettes/chep/).

As we made some project-level changes above, first we'll reset a few key project-level variables to make sure we're starting from fresh, i.e. including reattaching the original sample-list, and specifying that we'll only consider the first EEG (`EEG` or equivalently `EEG1`):

In [ ]:
proj.reset()
proj.sample_list( 'tutorial/s.lst' )
proj.var( 'sig', '.' )
proj.var( 'sig' , 'EEG' )

This script is contained in `fifth.txt`:

In [ ]:
%%sh
cat tutorial/cmd/fifth.txt

Here we'll run just for the first individual:

In [ ]:
p = proj.inst( 1 )
p.eval( lp.cmdfile( 'tutorial/cmd/fifth.txt' ) )

These are the mean Hjorth parameter values across all sleep epochs:

In [ ]:
p.table( 'SIGSTATS', 'CH' )

Now we'll add in additional commands to flag/remove epochs that are outliers, using `CHEP-MASK` and `CHEP` as described [here](https:/zzz.bwh.harvard.edu/luna/ref/artifacts/#chep-mask):

In [ ]:
%%sh
cat tutorial/cmd/sixth.txt

In [ ]:
p.eval( lp.cmdfile( 'tutorial/cmd/sixth.txt' ) )

To generate an equivalent plot as in the original tutorial, we'll pull out tables for the mask information (`tm`, was the epoch flagged as potential artifact or not) and epoch-level Hjorth parameters (`th`) and plot them:

In [ ]:
tm = p.table( 'DUMP_MASK' , 'E' )
th = p.table( 'SIGSTATS' , 'CH_E' )

In [ ]:
import numpy as np
cl = [ ['black','red'][e] for e in tm.EMASK ] 
sz = [ [1,5][e] for e in tm.EMASK ] 
fig, axs = plt.subplots(3, figsize=(10, 3) )
fig.suptitle('EEG Hjorth parameters and masking')
axs[0].scatter( th.E , th.H1 , s=sz, c = cl )
axs[1].scatter( th.E , th.H2 , s=sz, c = cl ) 
axs[2].scatter( th.E , th.H3 , s=sz, c = cl )
plt.show()

These plots are similar to those generated in the original tutorial.   We can also mirror the plotting of raw signal data, e.g. for epochs flagged as potentially bad (e.g. here epoch 220 in the original EDF, and 145 in the masked version):

In [ ]:
d = p.slice( p.e2i( 145 ) , chs = ['EEG' ] )
plt.plot( d[1] )
plt.show()

## Spectral & spindle analyses

Finally, we'll repeat some of the spectral and spindle analyses performed in the tutorial, from `seventh.txt`, running just for the first `EEG` channel.   This also shows how we can exclude epochs based on the type of epoch-level artifact detection methods above.

In [ ]:
proj.var('sig','')
proj.var('sig','EEG')

In [ ]:
%%sh
cat tutorial/cmd/seventh.txt

In [ ]:
res = proj.proc( lp.cmdfile( 'tutorial/cmd/seventh.txt' ) ) 

These are the output tables generated:

In [ ]:
proj.strata()

Here we pull out power spectra (metrics from `PSD` stratified by channel (`CH`) and frequency bin (`F`):

In [ ]:
t = proj.table( 'PSD' , 'CH_F' ) 
t

Plotting these, we see similar spectra as from the original tutorial:

In [ ]:
import numpy as np
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 3) )
fig.suptitle('NREM EEG power spectra')
ax1.plot( t[ t.ID == 'learn-nsrr01' ].F, 10*np.log10( t[ t.ID == 'learn-nsrr01' ].PSD.astype(float) ) )
ax2.plot( t[ t.ID == 'learn-nsrr02' ].F, 10*np.log10( t[ t.ID == 'learn-nsrr02' ].PSD.astype(float) ) )
ax3.plot( t[ t.ID == 'learn-nsrr03' ].F, 10*np.log10( t[ t.ID == 'learn-nsrr03' ].PSD.astype(float) ) )
plt.show()

Following the tutorial, we can repeat without any filtering to see the impact on the PSD: (here we'll include the second EEG channel also).  Note, as we use the results of the above run (in `res`) below, we'll save these output to a dummy object `xxx`.  Note that that internal cache from the above run will be over-written when running this. 

In [ ]:
proj.var( 'sig', 'EEG2' )
xxx = proj.silent_proc( ' EPOCH & MASK if=W & PSD spectrum max=62 ' )

In [ ]:
display( proj.strata() )
t = proj.table( 'PSD' , 'CH_F' ) 

In [ ]:
# plot all raw PSDs (rows = channels, cols = indivs)
fig, axs = plt.subplots(2, 3, figsize=(10, 3) )
fig.suptitle('NREM EEG power spectra')
chs = t.CH.unique()
ids = t.ID.unique()
t.PSD = t.PSD.astype(float)
for ch in list(range(0,2)):
    for id in list(range(0,3)):
        t1 = t[ ( t.ID == ids[id] ) & ( t.CH == chs[ch] ) ] 
        axs[ch][id].plot( t1.F, 10*np.log10( t1.PSD ) )


Finally, we can pull out the spindle results from the initial run (in `res`):

In [ ]:
t = res['SPINDLES: CH_F']
t

We extract spindle density (`DENS`) for individual for fast and slow spindles, getting values that match the original tutorial.

In [ ]:
pd.pivot( t , index = 'ID' , columns = 'F' , values='DENS' )

This concludes this tutorial for now.